In [1]:
import numpy as np
import tvm
from tvm import te

In [4]:
def conv_out_size(n,k,p,s):
    return (n-k+2*p)//s + 1
def padding(X, ph, pw, val=0):
    assert len(X.shape) >= 2
    nh,nw = X.shape[-2],X.shape[-1]
    return te.compute(
        (*X.shape[0:-2],nh+ph*2,nw+pw*2),
        lambda *i:te.if_then_else(
            te.any(i[-2]<ph, i[-2]>=nh+ph, i[-1]<pw, i[-1]>=nw+pw),
            val,X[i[:-2]+(i[-2]-ph,i[-1]-pw)]),name='PaddedX')

In [5]:
def get_conv_data(oc,ic,n,k,p=0,s=1,constructor=None,conv_type='direct'):
    np.random.seed(0)
    data = np.random.normal(size=(ic,n,n)).astype('float32')
    ic_weight = ic
    if conv_type == 'depthwise':
        ic_weight = 1
    weight = np.random.normal(size=(oc,ic,k,k)).astype('float32')
    on = conv_out_size(n,k,p,s)
    out = np.empty((oc,on,on), dtype='float32')
    if constructor:
        data, weight, out = (constructor(x) for x in [data, weight, out])
    return data,weight,out